# Report for 2D Project Physical World and Digital World

Cohort: # 07

Team No.: # 02

Members:
* Keith Goh Guan Da(1003334)
* Cheow Pak Leng Josiah (1003602)
* Yang Peng (1003776)
* Jessica Davinia Layardi (1003675)
* Xu Shuangyi (1002952)

# Introduction

Write your introduction here. Describe the problem you want to solve and a brief of your approach and summary of your result.


# Description of Data from Experiment

## Data Collection

Describe how you collect your data and the reasoning behind such approach.



We used an rpi to log the time and temperature of the water. We put the temperature probe in after a few seconds of starting the probe. We then ran the programme till the temperature is constant.

In [0]:
import os
import glob
import time
import csv
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'
 #function which reads each line of the file
def read_temp_raw():
    f = open(device_file, 'r')
    lines = f.readlines()
    f.close()
    return lines
 
def read_temp():
    lines = read_temp_raw()
    
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw()
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        return temp_c
    
try:
    with open('./tempreadingtest22.csv','a') as f:
        csv_out=csv.writer(f)
        csv_out.writerow(('Time','Temp'))
        while True:
            csv_out.writerow((time.strftime("%Y-%m-%d %H:%M:%S"),read_temp()))
            print(read_temp())
except KeyboardInterrupt:
    f.close()

IndexError: ignored

This code allows us to to read and update csv files with our time and temperature readings

## Data Preparation

Describe how you prepare your data. Include any codes you have here.

We first converted the csv file into xlsx format and also took away the excess data from the start and end of the csv files manually.
We then automated the process of converting the time into seconds and also processes the temperature data to the linear form.

## Data Format

Describe your data and its features. Include any codes or visualization of data.

The data is in the form $\ln\frac{\mathrm{T}_{\mathrm{w}}-\mathrm{T}_{\mathrm{s}}}{\mathrm{T}_{\mathrm{w}}-\mathrm{T}_{\mathrm{amb}}}$against time.


# Training Model

Describe how you train your model. Include any code and output

We split the data into training and test sets and used sk learn to linearly regress the data. We found a value of tau.

# Verification and Accuracy

Describe how you check the accuracy of your model and its result. State any analysis you have and the steps you have taken to improve its accuracy.

We tested the model using the model against the test data. The  $r^2$ value and MSE value was used to evaluate the data.
We took more data sets in order to improve the results. However we found that for different data sets they have different values of $\tau$ which is related to $-\frac{1}{gradient}$.

# Example Scripts

Instruction:

* Read an excel file with the following format:
```
time (s)	reading
0.00	    25.812
0.90	    28.562
1.79	    31.875
2.68	    35.062
3.55	    37.937
4.43	    40.687
5.30	    43.25
```
where the first column indicates the time in seconds and the second column indicates the sensor reading in Celsius. 
* Write a code to prepare the data, extract the features.
* Write a code to split the data.
* Write a code to train the model.
* Write a code to predict the final temperature.
* Write a code to check accuracy.

**The script below is just for your example. You don't have to use it and you can write your own script.**



First we started to import the data into panda dataframe

In [0]:
# write a code to read an excel file

import pandas as pd

# specify the base of your filename, e.g. temp_1.xlsx, temp_2.xlsx
filename = 'tempreadingtest' 

# if you have more than one files, 
# you can use some key to differentiate them, e.g. '1', '2'
filekeys = ['','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'] 

# this is to store the data for different files, 
# the keys are in filekeys
dataframe = {} 
for key in filekeys:
    dataframe[key] = pd.read_excel(filename + key + '.xlsx')


We then prepared the data into the dictionary data processed with different keys for different sets of data. With one column being time in seconds since the start of the experiment and the other column being  $\ln\frac{\mathrm{T}_{\mathrm{w}}-\mathrm{T}_{\mathrm{s}}}{\mathrm{T}_{\mathrm{w}}-\mathrm{T}_{\mathrm{amb}}}$

In [0]:
# write a code to prepare the data for predicting
import numpy as np
def preprocess(df):
    # use this function to extract the features from the data frame
    minutestore=''
    timearray=np.array([])
    temparray=np.array([])
    #appointing temperature data from our frame
    Tempw=df["Temp"].iloc[-1]
    Tempamb=df["Temp"].iloc[0]
    #loop through each row
    for index, row in df.iterrows():
      #we want to split the minutes and seconds
      timenow=str(row['Time'])
      splitdate=timenow.split(' ')[1]
      seconds=int(splitdate.split(':')[2])
      minutes=int(splitdate.split(':')[1])
      #storing of minutes to be converted into seconds
      if minutestore=='':
        minutestore=minutes
      #storing of seconds
      if minutes-minutestore>0:
        seconds=(minutes-minutestore)*60+seconds
      elif minutestore-minutes>0:
        seconds=(60-(minutestore-minutes))*60+seconds
      if (row['Temp']-Tempw)==0:
        pass
      else:
        #storing temperature and time data into array
        temparraynew=np.array([[np.log((row['Temp']-Tempw)/(Tempamb-Tempw))]])
        timearraynew=np.array([[seconds]])
        
        if timearray.size==0:
          timearrayfirst=timearraynew
          timearray=np.array([[0]])
          
        else:
          timearraynew-=timearrayfirst
          timearray=np.concatenate((timearray,timearraynew))
        if temparray.size==0:
          temparray=temparraynew
          
        else:
          temparray=np.concatenate((temparray,temparraynew))
    return timearray,temparray      


data_processed = {}
for key in filekeys:
    data_processed[key]=preprocess(dataframe[key])


We then split the data and combined all the data in the various datasets into one full dataset the split it into training and test sets.

In [0]:

# write a code to split the data to train and test
from sklearn.model_selection import train_test_split


def prepare_train_test(data,filekey):
  totaltime=np.array([[0]])
  totaltemp=np.array([[0]])
  for keys in filekey:
    if np.size(totaltemp)==0:
      totaltime=data[keys][0]
      totaltemp=data[keys][1]
    else:
      totaltime=np.concatenate((totaltime,data[keys][0]))
      totaltemp=np.concatenate((totaltemp,data[keys][1]))
  x_train,x_test,y_train,y_test=train_test_split(totaltime,totaltemp,test_size=0.3,random_state=5)
  traindata=(x_train,y_train)
  testdata=(x_test,y_test)
  return traindata,testdata

    

data_train, data_test = prepare_train_test(data_processed,filekeys)

We then used a linear model to fit data.

In [0]:
from sklearn import linear_model 

# write a code to train the model
# the function should return the trained model
def train_model(data):
  regr=linear_model.LinearRegression()
  model=regr.fit(data[0],data[1])
  c1=regr.coef_
  c0=regr.intercept_
  return model
  


model = train_model(data_train)

We used the mean square error and r2 score to evaluate the data.

In [0]:
from sklearn.metrics import mean_squared_error, r2_score

def tester(regr,data):    
    c0=regr.intercept_
    c1=regr.coef_
    y_pred=regr.predict(data[0])
    MSE=mean_squared_error(y_pred,data[1])
    R2=r2_score(data[1],y_pred)
    results={}
    results['coefficients']=c1
    results['intercept']=c0
    results['mean squared error']=MSE
    results['r2 score']=R2
    results['tau']=-1/c1[0][0]
    return results
print(tester(model,data_test))

{'coefficients': array([[-0.05278939]]), 'intercept': array([-0.25084623]), 'mean squared error': 0.29289079716707433, 'r2 score': 0.8952962273743904, 'tau': 18.943202096637233}


Since the results were not good, we tried to train individual models for individual data sets to test how the value of tau would change for each set.

In [0]:
# write a code to prepare the data for predicting
import numpy as np
def preprocess(df):
    # use this function to extract the features from the data frame
    minutestore=''
    timearray=np.array([])
    temparray=np.array([])
    #appointing temperature data from our frame
    Tempw=df["Temp"].iloc[-1]
    Tempamb=df["Temp"].iloc[0]
    #loop through each row
    for index, row in df.iterrows():
      #we want to split the minutes and seconds
      timenow=str(row['Time'])
      splitdate=timenow.split(' ')[1]
      seconds=int(splitdate.split(':')[2])
      minutes=int(splitdate.split(':')[1])
      #storing of minutes to be converted into seconds
      if minutestore=='':
        minutestore=minutes
      #storing of seconds
      if minutes-minutestore>0:
        seconds=(minutes-minutestore)*60+seconds
      elif minutestore-minutes>0:
        seconds=(60-(minutestore-minutes))*60+seconds
      if (row['Temp']-Tempw)==0:
        pass
      else:
        #storing temperature and time data into array
        temparraynew=np.array([[np.log((row['Temp']-Tempw)/(Tempamb-Tempw))]])
        timearraynew=np.array([[seconds]])
        
        if timearray.size==0:
          timearrayfirst=timearraynew
          timearray=np.array([[0]])
          
        else:
          timearraynew-=timearrayfirst
          timearray=np.concatenate((timearray,timearraynew))
        if temparray.size==0:
          temparray=temparraynew
          
        else:
          temparray=np.concatenate((temparray,temparraynew))
    return timearray,temparray      


data_processed = {}
for key in filekeys:
    data_processed[key]=preprocess(dataframe[key])

In [0]:

# write a code to split the data to train and test
from sklearn.model_selection import train_test_split


def prepare_train_test(data):
  
  x_train,x_test,y_train,y_test=train_test_split(data[0],data[1],test_size=0.3,random_state=5)
  traindata=(x_train,y_train)
  testdata=(x_test,y_test)
  return traindata,testdata

    
datatrain={}
datatest={}
for key in filekeys:
  datatrain[key], datatest[key] = prepare_train_test(data_processed[key])


In [0]:
from sklearn import linear_model 
# write a code to train the model
# the function should return the trained model
def train_model(data):
  regr=linear_model.LinearRegression()
  model=regr.fit(data[0],data[1])
  c1=regr.coef_
  c0=regr.intercept_
  K=(-1/c1)[0][0]
  return model
  

model={}
for key in filekeys:
  model[key] = train_model(datatrain[key])

In [0]:
# write a code to predict the final temperature
# store the predicted temperature in a variable called "predicted"
# predicted is a dictionary where the keys are listed in filekey
from sklearn.metrics import mean_squared_error, r2_score

def tester(regr,data):    
    c0=regr.intercept_
    c1=regr.coef_
    y_pred=regr.predict(data[0])
    MSE=mean_squared_error(y_pred,data[1])
    R2=r2_score(data[1],y_pred)
    results={}
    results['coefficients']=c1
    results['intercept']=c0
    results['mean squared error']=MSE
    results['r2 score']=R2
    results['y_pred']=y_pred
    results['tau']=-1/c1[0][0]
    return results
  
predicted = {}  
resultsdict={}
for key in filekeys:
    resultsdict[key]=tester(model[key],datatest[key])
    predicted[key]=resultsdict[key]['y_pred']
    
    
    